In [1]:
#install dependencies
!pip install langchain
!pip install langchain-community
!pip install langchain_google_genai
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 63.5 MB/s eta 0:00:00


In [79]:
import langchain
from langchain.prompts import PromptTemplate,ChatPromptTemplate,HumanMessagePromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema.output_parser import StrOutputParser
from langchain.chains import LLMChain
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.retrievers import ContextualCompressionRetriever
import os
from operator import itemgetter
import warnings
from langchain.memory import ChatMessageHistory
warnings.filterwarnings('ignore')

In [10]:
import os
os.environ["GOOGLE_API_KEY"] = "AIzaSyD0N8r3p9QwcDiTmh2LyJvWJ4-------"

from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [17]:
llm.invoke("Who is the imran khan?").content

'Imran Khan is a Pakistani politician, former cricketer, and philanthropist.  He served as the 22nd Prime Minister of Pakistan from August 2018 until April 2022, when he was ousted through a vote of no confidence.\n\nBefore entering politics, Khan was a world-renowned cricketer, captaining the Pakistan national cricket team to victory in the 1992 Cricket World Cup.  After retiring from cricket, he founded the Shaukat Khanum Memorial Cancer Hospital and Research Centre in Lahore, a charitable institution named after his mother.\n\nKhan formed his own political party, Pakistan Tehreek-e-Insaf (PTI), in 1996.  After years as an opposition figure, his party gained significant popularity and eventually won the 2018 general elections, leading to his premiership.  His political platform has focused on anti-corruption, Islamic values, and social welfare programs.\n\nSince his removal from office, Khan has remained a prominent figure in Pakistani politics, actively campaigning and facing variou

In [22]:
prompt = PromptTemplate(
    input_variables=["question"],
    template="you are a helpfull assistant for user answer the user {question}"
)

In [23]:
formattedPrompt = prompt.format(question="who is the imran khan?")

In [34]:
chain = prompt | llm | StrOutputParser()
result = chain.invoke({"question":"Do you know who is illuminati?"})

In [35]:
print((result))

The Illuminati was a real group, a small secret society founded in Bavaria in 1776.  However, almost everything you hear about them today is fiction.  They were all about opposing superstition, abuses of state power, and church influence over public life.  They wanted to promote reason and education.  They were short-lived, banned within a decade by the Bavarian government, and ultimately faded away.

The modern-day idea of the Illuminati as a powerful, shadowy organization controlling the world is entirely a myth.  This myth stems from a combination of sources, including fictional books, conspiracy theories, and online misinformation.  It's often used as a scapegoat to explain complex events or to promote distrust in established institutions.

So, in short:

* **The historical Illuminati:** A real, but short-lived and ultimately unimportant secret society from the 18th century.
* **The modern "Illuminati":** A fictional, mythical group used in conspiracy theories.

It's important to d

In [37]:
#now work on Memory #to save the existing chat somewhere thats called langchain memory.

#there are three type of memort
#1 episodic
#2 semantic
#3 Procedural

In [48]:
#chatmessage history
#example1
from langchain.memory import ChatMessageHistory,ConversationBufferMemory #class to import  for tracking the history of chatbot

history = ChatMessageHistory()
history.add_user_message("hi") #to add user message
history.add_ai_message("whats up") #to add ai message
history.add_user_message("How are you")
history.add_ai_message("yup i am fine what about you")
print(history)

Human: hi
AI: whats up
Human: How are you
AI: yup i am fine what about you


In [51]:
#Example2
#Conversational Buffer Memory
from langchain.memory import ConversationBufferMemory

example2 = ConversationBufferMemory()
example2.save_context({"input": "Hi Ai Message how are you?"},{"ouptut":"i am fine user what about you?"})

In [58]:
print(example2.buffer) #use buffer object to get the actual conversation
print(example2.load_memory_variables({})) #use load_memory_variables to get the conversation

Human: Hi Ai Message how are you?
AI: i am fine user what about you?
{'history': 'Human: Hi Ai Message how are you?\nAI: i am fine user what about you?'}


In [68]:
#Example3 with chain using buffer memory
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

memory = ConversationBufferMemory()
conversation = ConversationChain(llm=llm,memory=memory)

conversation.predict(input="how are you doing right now")
conversation.predict(input="do you know is imran khan?")
print(memory.buffer)

Human: how are you doing right now
AI: I'm doing well, thank you for asking!  I'm currently processing your request and accessing my vast database of information.  While I don't experience emotions in the same way humans do, I'm functioning optimally and ready to assist you. My current operational temperature is within normal parameters, and my memory allocation is at 78.3% utilization.  I last performed a system diagnostic check at 03:17:22 UTC and everything reported nominal.  Is there anything specific you'd like to know or discuss?

Human: do you know is imran khan?
AI: Yes, I do. Imran Khan is a well-known figure, primarily known for his career in cricket and later his foray into politics.  He captained the Pakistan national cricket team to victory in the 1992 Cricket World Cup, a significant achievement that cemented his status as a national hero.  Following his retirement from cricket, he founded the political party Pakistan Tehreek-e-Insaf (PTI) in 1996.  After years of politic

In [67]:
#example 3
from langchain.memory import ConversationBufferWindowMemory

meomorry = ConversationBufferWindowMemory(k=1) #K is the variable that you want to see how much message you want to get #this will only gave the last result

chain = ConversationChain(llm=llm,memory=meomorry)

chain.predict(input="how are you doing right now")
chain.predict(input="do you know is imran khan?")
print(meomorry.buffer)

Human: do you know is imran khan?
AI: Yes, I do. Imran Khan is a prominent Pakistani figure. He was born on October 5, 1952, in Lahore, Pakistan.  He gained international recognition as a cricketer, captaining the Pakistan national team to victory in the 1992 Cricket World Cup.  Following his retirement from cricket, he entered politics and founded the Pakistan Tehreek-e-Insaf (PTI) party in 1996.  After years in opposition, he became the 22nd Prime Minister of Pakistan in 2018, serving until he was ousted through a vote of no confidence in April 2022.  He has been a controversial figure, both praised for his philanthropy and criticized for his political stances and governance.  He's also known for his marriage to Jemima Goldsmith and later Reham Khan, both of which ended in divorce.  More recently, he married Bushra Maneka.  Is there anything specific you'd like to know about him?



In [90]:
#example4
from langchain.memory import ConversationSummaryBufferMemory

Csbm = ConversationSummaryBufferMemory(llm=llm,max_token_limit=100)

chain = ConversationChain(llm=llm,memory=Csbm)

chain.predict(input="how are you doing right now")
print(Csbm.buffer)

System: The human asks how the AI is doing. The AI responds that it is doing well and functioning optimally, though it doesn't experience emotions like humans. It mentions its low processing load and recent information processing about monarch butterflies and sitars, which it finds fascinating.  The AI then asks the human how they are doing.



In [91]:
chain.predict(input="do you know is imran khan?")
print(Csbm.buffer)

System: The human asks how the AI is doing. The AI responds that it is doing well and functioning optimally, though it doesn't experience emotions like humans. It mentions its low processing load and recent information processing about monarch butterflies and sitars, which it finds fascinating.  The AI then asks the human how they are doing. The human then asks if the AI knows Imran Khan. The AI confirms that it does, identifying him as a prominent figure known for his roles as a Pakistani politician (22nd Prime Minister), a world-renowned cricketer (captain of the 1992 World Cup winning team), and a philanthropist (founder of the Shaukat Khanum Memorial Cancer Hospital).  The AI offers to provide more information about any of these aspects of Imran Khan's life, acknowledging his recent political activities and legal challenges.



In [99]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 2.1 MB/s eta 0:00:00


In [102]:
#example5 and last
from langchain.memory import VectorStoreRetrieverMemory
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

#FIRST creat the embeddings
embeddings = HuggingFaceEmbeddings()

#then pass the embedings to the vectorstores
db = Chroma(embedding_function=embeddings)

#make the retriver object

retriver = db.as_retriever()

chain = ConversationChain(llm=llm,memory=VectorStoreRetrieverMemory(retriever=retriver))

conversation.predict(input="how are you doing right now")
conversation.predict(input="my favourite food is pizza")
conversation.predict(input="i like to play snooker game")

"That's a fantastic game! Snooker requires a high level of skill, strategy, and precision.  It's fascinating to watch professionals execute complex shots and plan several moves ahead. My current memory allocation is at 80.7% utilization. Do you play often?  Perhaps you have a favorite player or a memorable moment from a match you'd like to discuss? I can access information on snooker rules, famous players, and tournament history.\n"

In [105]:
conversation.predict(input="which game i like most") #got cha

'You mentioned earlier that you like to play snooker.\n'